# Введение в декораторы

In [1]:
def my_function():
    print('I am a simple function')

In [2]:
def my_decorator(f):
    def wrapper_function(*args, **kwargs):
        """I am wrapper function"""
        print('DECORATOR:', f.__name__, args, kwargs)
        return f(*args, **kwargs)
    return wrapper_function

In [3]:
@my_decorator
def my_function():
    print('I am a simple function')

In [4]:
@my_decorator
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [5]:
identity_function(4)

DECORATOR: identity_function (4,) {}
FUNCTION


4

## Параметры декорируемой функции

In [6]:
identity_function.__name__

'wrapper_function'

In [7]:
identity_function.__doc__

'I am wrapper function'

In [8]:
identity_function.__module__

'__main__'

Способ 1:

In [41]:
def my_decorator(f):
    def wrapper_function(*args, **kwargs):
        """I am wrapper function"""
        print('DECORATOR:', f.__name__, args, kwargs)
        return f(*args, **kwargs)
    wrapper_function.__module__ = f.__module__
    wrapper_function.__name__ = f.__name__
    wrapper_function.__doc__ = f.__doc__
    return wrapper_function

In [42]:
@my_decorator
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [43]:
identity_function.__name__

'identity_function'

Способ 2:

In [191]:
def update_wrapper(function1, function2):
    function1.__module__ = function2.__module__
    function1.__name__ = function2.__name__
    function1.__doc__ = function2.__doc__
    return function1

In [192]:
def my_decorator(f):
    def wrapper_function(*args, **kwargs):
        """I am wrapper function"""
        print('DECORATOR:', f.__name__, args, kwargs)
        return f(*args, **kwargs)
    return update_wrapper(wrapper_function, f)

In [193]:
@my_decorator
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [189]:
identity_function.__name__

'identity_function'

Способ 3:

In [194]:
def update_wrapper(function1, function2):
    function1.__module__ = function2.__module__
    function1.__name__ = function2.__name__
    function1.__doc__ = function2.__doc__
    return function1

In [200]:
def my_decorator(f):
    specific_update_wrapper = lambda x: update_wrapper(x, f)
    
    @specific_update_wrapper
    def wrapper_function(*args, **kwargs):
        """I am wrapper function"""
        print('DECORATOR:', f.__name__, args, kwargs)
        return f(*args, **kwargs)
    return wrapper_function

In [201]:
@my_decorator
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [202]:
identity_function.__name__

'identity_function'

Способ 4:

In [218]:
import functools

In [219]:
def my_decorator(f):
    @functools.wraps(f)
    def wrapper_function(*args, **kwargs):
        """I am wrapper function"""
        print('DECORATOR:', f.__name__, args, kwargs)
        return f(*args, **kwargs)
    return wrapper_function

In [220]:
@my_decorator
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [222]:
identity_function.__name__

'identity_function'

## Управление состоянием декоратора

In [241]:
def my_decorator(f):
    @functools.wraps(f)
    def wrapper_function(*args, **kwargs):
        """I am wrapper function"""
        print('DECORATOR:', f.__name__, args, kwargs)
        return f(*args, **kwargs)
    return wrapper_function if my_decorator.flag else f

my_decorator.flag = False

In [242]:
@my_decorator
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [243]:
identity_function(8)

FUNCTION


8

## декоратор с аргументами

In [256]:
def my_decorator_with_args(useless_string):
    def my_decorator(f):
        @functools.wraps(f)
        def wrapper_function(*args, **kwargs):
            print('DECORATOR:', useless_string)
            return f(*args, **kwargs)
        return wrapper_function
    return my_decorator

In [257]:
@my_decorator_with_args('i want to be printed')
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [258]:
identity_function(15)

DECORATOR: i want to be printed
FUNCTION


15

In [331]:
def my_decorator_with_args(useless_string):
    print("Я создаю декоратор.")
    def my_decorator(f):
        print("Я - декоратор.")
        def wrapper_function(*args, **kwargs):
            print(useless_string)
            return f(*args, **kwargs)
        print("Я возвращаю обёрнутую функцию.")
        return wrapper_function
    print("Я возвращаю декоратор.")
    return my_decorator


In [332]:
@my_decorator_with_args('Печатаюсь внутри декоратора')
def identity_function(x):
    print("Я функция, мой аргумент - {}".format(x))
    return x

Я создаю декоратор.
Я возвращаю декоратор.
Я - декоратор.
Я возвращаю обёрнутую функцию.


In [329]:
identity_function(16)

Печатаюсь внутри декоратора
Я функция, мой аргумент - 16


16

In [330]:
identity_function(23)

Печатаюсь внутри декоратора
Я функция, мой аргумент - 23


23

более универсальный, но замороченный способ

задекорируем функцию, которая конструирует декоратор

In [435]:
def with_arguments(deco_with_args):
    @functools.wraps(deco_with_args)
    def deco_without_args(*args, **kwargs):
        return lambda f: deco_with_args(f, *args, **kwargs)
    return deco_without_args

In [436]:
@with_arguments
def my_decorator_with_args(f, useless_string):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        print('DECORATOR:', useless_string)
        return f(*args, **kwargs)
    return wrapper

In [443]:
@my_decorator_with_args('i want to be printed')
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [444]:
identity_function(42)

DECORATOR: i want to be printed
FUNCTION 42


Ещё более короткая запись:

In [457]:
def with_arguments(deco_with_args):
    @functools.wraps(deco_with_args)
    def deco_without_args(*args, **kwargs):
        def wrapper(f):
            result = deco_with_args(f, *args, **kwargs)
            functools.update_wrapper(result, f)
            return result
        return wrapper
    return deco_without_args

In [460]:
@with_arguments
def my_decorator_with_args(f, useless_string):
    def wrapper(*args, **kwargs):
        print('DECORATOR:', useless_string)
        return f(*args, **kwargs)
    return wrapper

In [463]:
@my_decorator_with_args('i want to be printed')
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [464]:
identity_function.__name__

'identity_function'

Аргументы по-умолчанию:

In [519]:
def with_arguments(deco_with_args):
    @functools.wraps(deco_with_args)
    def deco_without_args(*args, **kwargs):
        def wrapper(f):
            result = deco_with_args(f, *args, **kwargs)
            functools.update_wrapper(result, f)
            return result
        return wrapper
    return deco_without_args

In [520]:
@with_arguments
def my_decorator_with_args(f, useless_string='Nothing'):
    def wrapper(*args, **kwargs):
        print('DECORATOR:', useless_string)
        return f(*args, **kwargs)
    return wrapper

In [521]:
@my_decorator_with_args
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [522]:
identity_function(153)

<function __main__.my_decorator_with_args.<locals>.wrapper>

In [530]:
identity_function(lambda x: print('WHAT IS'))(153)

DECORATOR: <function identity_function at 0x7ff87441dea0>
WHAT IS


In [487]:
@my_decorator_with_args()
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [489]:
identity_function(153)

DECORATOR: Nothing
FUNCTION


153

2 способ писать декораторы с аргументами:

In [541]:
def my_decorator_with_args(f=None, *, useless_string='Nothing'):
    # со скобками
    if f is None:
        return lambda f: my_decorator_with_args(f, useless_string=useless_string)

    # без скобок
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        print('DECORATOR:', useless_string)
        return f(*args, **kwargs)
    return wrapper

In [535]:
@my_decorator_with_args
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [536]:
identity_function(153)

DECORATOR: Nothing
FUNCTION


153

In [537]:
@my_decorator_with_args(useless_string='i want to be printed')
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [538]:
identity_function(153)

DECORATOR: i want to be printed
FUNCTION


153

3 способ писать декораторы с аргументами

In [605]:
class my_decorator_with_args:
    def __init__(self, useless_string='Nothing'):
        self.useless_string = useless_string
        
    def __call__(self, f):
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            print('DECORATOR:', self.useless_string)
            return f(*args, **kwargs)
        return wrapper

In [579]:
@my_decorator_with_args()
def identity_function(x):
    """I am identity function"""
    print('FUNCTION')
    return x

In [577]:
identity_function(153)

DECORATOR: Nothing
FUNCTION


153

## функция с обязательными ключевыми аргументами

In [51]:
def f(f=None, *, key1=0, key2=1):
    print(f)

In [53]:
f(key1=1, key2=2)

None


## декораторы классов

In [606]:
class MyClass:
    def __init__(self, x):
        self.x = x
    
    @my_decorator_with_args(useless_string='i want to be...')
    def square(self):
        return self.x ** 2

In [607]:
t = MyClass(1)

In [608]:
t.square()

DECORATOR: i want to be...


1

## конкретные параметры

подсчёт среднего времени выполнения

In [42]:
import time
import numpy as np
import functools

In [627]:
def time_count(func=None, *, n_iter=100):
    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")
        total_time = 0
        for i in range(n_iter):
            tick = time.clock()
            result = func(*args, **kwargs)
            total_time += time.clock() - tick
        print("mean time is {}".format(total_time / n_iter))
        return result
    return inner

In [628]:
@time_count
def count_nonzero_elements(x):
    return np.sum(x != 0)

In [631]:
count_nonzero_elements(np.random.randint(0, 2, size=1000))

count_nonzero_elements ... mean time is 1.8840000000039935e-05


496

In [43]:
def profiled(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    inner.ncalls = 0
    return inner

@profiled
def identity_function(x):
    return x

In [642]:
for i in range(10):
    identity_function(1)

In [643]:
identity_function.ncalls

10

мемоизация

In [13]:
import functools

In [19]:
def memoized(func):
    cache = {}
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))
        print(cache)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner

In [23]:
@memoized
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)
    else:
        return ackermann(m - 1, ackermann(m, n - 1))

In [24]:
ackermann(2, 1)

{}
{}
{}
{}
{}
{(0, 1): 2, (1, 0): 2}
{(0, 1): 2, (2, 0): 3, (1, 0): 2, (0, 2): 3, (1, 1): 3}
{(0, 1): 2, (2, 0): 3, (1, 0): 2, (0, 2): 3, (1, 1): 3}
{(0, 1): 2, (2, 0): 3, (1, 0): 2, (0, 2): 3, (1, 1): 3}
{(0, 1): 2, (2, 0): 3, (1, 0): 2, (0, 2): 3, (1, 1): 3}
{(0, 1): 2, (2, 0): 3, (1, 2): 4, (1, 0): 2, (0, 2): 3, (0, 3): 4, (1, 1): 3}


5

депрекейтед:

In [37]:
import warnings

def deprecated(func):
    code = func.__code__
    warnings.warn(func.__name__ + " is deprecated.")
    return func


In [38]:
@deprecated
def identity_function(x):
    return x

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: identity_function is deprecated.
  """


In [55]:
@np.vectorize
def my_func(x):
    if x > 5:
        return 25
    else:
        return x ** 2


In [56]:
data = np.array([1, 2, 3, 4, 5, 6])
my_func(data)

array([ 1,  4,  9, 16, 25, 25])

array([ 1,  4,  9, 16, 25, 25])